In [1]:
base_path = "/content/drive/My Drive/Github/mtc-device-activation"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from casbl import casbl

data = np.load(f'{base_path}/general_model/data/general_model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load(f'{base_path}/communication_model/data/communication_simulation_theta.npz', allow_pickle=True)
all_theta = data["all_theta"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_gamma.npz', allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_z.npz', allow_pickle=True)
all_z = data["all_z"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_noise.npz', allow_pickle=True)
all_noise = data["all_noise"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_received_signal.npz', allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_general.npz', allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

max_iter = 500

Mounted at /content/drive


In [4]:
sample_range = range(0, 200)
num_samples = len(sample_range)

Sparse Bayesian Learning (SBL)

In [5]:
# Final gamma and z results
gamma_result_all = np.empty((num_samples, len(pilot_length_array)), dtype=object)
z_result_all = np.empty((num_samples, len(pilot_length_array)), dtype=object)

In [ ]:
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    z = all_z[sample_index]

    for pilot_length_index in tqdm(range(len(pilot_length_array)), desc="Processing Pilot Length Levels", leave=False, position=1):
        pilot_length = pilot_length_array[pilot_length_index]

        theta = all_theta[sample_index, pilot_length_index, :pilot_length]
        received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]

        # Compute noise
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
        noise_power = signal_power / snr

        # Run CASBL
        gamma_result, z_result, gamma_history, z_history, iteration_count = casbl(
            theta, received_signal, noise_power, loc, max_iter=max_iter
        )

        # Save outputs
        gamma_result_all[sample_index-sample_range[0], pilot_length_index] = gamma_result.copy()
        z_result_all[sample_index-sample_range[0], pilot_length_index] = z_result.copy()

Processing Pilot Length Levels:  67%|██████▋   | 4/6 [01:11<00:34, 17.15s/it]

Converged after 406 iterations



Processing Samples:   0%|          | 1/200 [01:51<6:08:27, 111.09s/it]

Converged after 469 iterations



Processing Pilot Length Levels:  67%|██████▋   | 4/6 [01:10<00:35, 17.63s/it]

...


Processing Samples:  32%|███▏      | 64/200 [1:57:39<3:59:37, 105.72s/it]

Converged after 281 iterations



Processing Samples:  32%|███▎      | 65/200 [1:59:37<4:06:08, 109.40s/it]

Converged after 424 iterations



Processing Pilot Length Levels:  17%|█▋        | 1/6 [00:16<01:24, 16.97s/it]

In [ ]:
np.savez_compressed(
    f'{base_path}/results_evaluation/data/casbl_snr_db_12_pilot_length_all_set_1.npz',
    gamma_result_all=gamma_result_all,
    z_result_all=z_result_all,
)

print("All results have been saved")